In [12]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

### Getting the Data

In [13]:
today = datetime.today().strftime('%Y-%m-%d')
start_date = '2016-01-01'
eth_df = yf.download('ETH-USD',start_date, today) #download of ethereum data using yfinance

[*********************100%***********************]  1 of 1 completed


### Checking for null data

In [14]:
eth_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1992 entries, 2017-11-09 to 2023-04-23
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       1992 non-null   float64
 1   High       1992 non-null   float64
 2   Low        1992 non-null   float64
 3   Close      1992 non-null   float64
 4   Adj Close  1992 non-null   float64
 5   Volume     1992 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 108.9 KB


In [15]:
eth_df.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

### Adjusting the column indexes

In [16]:
eth_df.head(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-11-09,$308.64,$329.45,$307.06,$320.88,$320.88,893249984
2017-11-10,$320.67,$324.72,$294.54,$299.25,$299.25,885985984
2017-11-11,$298.59,$319.45,$298.19,$314.68,$314.68,842300992
2017-11-12,$314.69,$319.15,$298.51,$307.91,$307.91,1613479936
2017-11-13,$307.02,$328.42,$307.02,$316.72,$316.72,1041889984
2017-11-14,$316.76,$340.18,$316.76,$337.63,$337.63,1069680000
2017-11-15,$337.96,$340.91,$329.81,$333.36,$333.36,722665984
2017-11-16,$333.44,$336.16,$323.61,$330.92,$330.92,797254016
2017-11-17,$330.17,$334.96,$327.52,$332.39,$332.39,621732992


In [17]:
eth_df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [18]:
#It lacks the "date" index, fixing it:
eth_df.reset_index(inplace=True)
eth_df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

### Choosing the right columns

In [21]:
#Prothet only needs two columns, the time (in this case, the date) and the parameter to be analyzed, in this case "open":

df = eth_df[['Date','Open']]

new_indexes = {
    'Date':'ds',
    'Open':'y'
}
df.rename(columns = new_indexes, inplace = True)
df.head(10)

,ds,y
0,2017-11-09,$308.64
1,2017-11-10,$320.67
2,2017-11-11,$298.59
3,2017-11-12,$314.69
4,2017-11-13,$307.02
5,2017-11-14,$316.76
6,2017-11-15,$337.96
7,2017-11-16,$333.44
8,2017-11-17,$330.17
9,2017-11-18,$331.98


### Using plotly library to plot the graph

In [27]:
# plot the open price
x = df["ds"]
y = df["y"]
fig = go.Figure() #Creates blank graph
fig.add_trace(go.Scatter(x=x, y=y))
# Set title
fig.update_layout(
    title_text="Time series plot of Ethereum Open Price",)

### Building Prophet Model

In [28]:
m = Prophet(
    seasonality_mode="multiplicative" 
)
m.fit(df)

15:35:16 - cmdstanpy - INFO - Chain [1] start processing
15:35:16 - cmdstanpy - INFO - Chain [1] done processing


In [29]:
m

##### The reason we set seasonality mode to “multiplicative” is we can assume it’s a multiplicative time series because of how cryptocurrency price fluctuates by the year, which also means the seasonal component changing with trend. More about Time Series: https://dziganto.github.io/python/time%20series/Introduction-to-Time-Series/


In [34]:
#Now we create a full year future dataframe, so we can extrapolate the data:
future = m.make_future_dataframe(periods = 365)
future

,ds
0,2017-11-09
1,2017-11-10
2,2017-11-11
3,2017-11-12
4,2017-11-13
...,...
2352,2024-04-18
2353,2024-04-19
2354,2024-04-20
2355,2024-04-21


In [36]:
# Using the prediction function:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2352,2024-04-18,$357.50,"$-1,877.08","$2,833.20"
2353,2024-04-19,$353.86,"$-1,964.44","$2,875.28"
2354,2024-04-20,$349.64,"$-1,911.37","$2,841.84"
2355,2024-04-21,$348.38,"$-1,954.68","$2,917.28"
2356,2024-04-22,$346.76,"$-1,980.10","$2,877.94"


### Ploting the Forecast

In [38]:
plot_plotly(m, forecast)

In [39]:
plot_components_plotly(m, forecast)